### Importing the Libraries used

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score
import numpy as np
from sklearn.ensemble import VotingClassifier

###  A bit of EDA

In [2]:
train= pd.read_csv("Train.csv")
test= pd.read_csv("test.csv")

In [3]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56000 entries, 0 to 55999
Data columns (total 52 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Applicant_ID    56000 non-null  object 
 1   form_field1     53471 non-null  float64
 2   form_field2     52156 non-null  float64
 3   form_field3     55645 non-null  float64
 4   form_field4     55645 non-null  float64
 5   form_field5     55645 non-null  float64
 6   form_field6     42640 non-null  float64
 7   form_field7     50837 non-null  float64
 8   form_field8     42640 non-null  float64
 9   form_field9     47992 non-null  float64
 10  form_field10    55645 non-null  float64
 11  form_field11    24579 non-null  float64
 12  form_field12    46105 non-null  float64
 13  form_field13    50111 non-null  float64
 14  form_field14    56000 non-null  int64  
 15  form_field15    33525 non-null  float64
 16  form_field16    42964 non-null  float64
 17  form_field17    44849 non-null 

In [4]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24000 entries, 0 to 23999
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Applicant_ID  24000 non-null  object 
 1   form_field1   22890 non-null  float64
 2   form_field2   22291 non-null  float64
 3   form_field3   23854 non-null  float64
 4   form_field4   23854 non-null  float64
 5   form_field5   23854 non-null  float64
 6   form_field6   18396 non-null  float64
 7   form_field7   21769 non-null  float64
 8   form_field8   18396 non-null  float64
 9   form_field9   20600 non-null  float64
 10  form_field10  23853 non-null  float64
 11  form_field11  10602 non-null  float64
 12  form_field12  19817 non-null  float64
 13  form_field13  21537 non-null  float64
 14  form_field14  24000 non-null  int64  
 15  form_field15  14408 non-null  float64
 16  form_field16  18526 non-null  float64
 17  form_field17  19305 non-null  float64
 18  form_field18  19631 non-nu

In [5]:
train.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50,default_status
0,Apcnt_1000000,3436.0,0.28505,1.6560,0.0,0.000,0.0,10689720.0,252072.0,4272776.0,...,0.392854,2.02,0.711632,0.0,0.0,charge,NaN,1.129518,0.044335,no
1,Apcnt_1000004,3456.0,0.67400,0.2342,0.0,0.000,0.0,898979.0,497531.0,9073814.0,...,0.314281,8.08,0.183584,NaN,0.0,charge,349.805730,1.620483,0.322436,no
2,Apcnt_1000008,3276.0,0.53845,3.1510,0.0,6.282,NaN,956940.0,NaN,192944.0,...,0.162965,18.18,0.791136,0.0,0.0,charge,NaN,1.513370,0.011640,yes
3,Apcnt_1000012,3372.0,0.17005,0.5050,0.0,0.000,192166.0,3044703.0,385499.0,3986472.0,...,0.488884,2.02,0.685168,NaN,0.0,charge,89.940100,0.664452,0.082729,no
4,Apcnt_1000016,3370.0,0.77270,1.1010,0.0,0.000,1556.0,214728.0,214728.0,1284089.0,...,0.275000,12.12,0.438168,0.0,0.0,charge,97.887502,1.427891,0.045630,no


In [6]:
test.head()

,Applicant_ID,form_field1,form_field2,form_field3,form_field4,form_field5,form_field6,form_field7,form_field8,form_field9,...,form_field41,form_field42,form_field43,form_field44,form_field45,form_field46,form_field47,form_field48,form_field49,form_field50
0,Apcnt_1000032,3236.0,0.34875,10.2006,0.0000,0.0,418564.0,418564.0,418564.0,540710.0,...,NaN,0.825000,1.01,0.800000,NaN,0.0,charge,NaN,0.000000,0.011221
1,Apcnt_1000048,3284.0,1.27360,2.9606,9.0198,0.0,0.0,9858816.0,49014.0,1510098.0,...,18.8415,0.507694,4.04,0.623248,1.0,0.0,lending,NaN,0.504974,0.043525
2,Apcnt_1000052,NaN,0.27505,0.0600,0.0000,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,0.00,NaN,NaN,NaN,charge,NaN,0.000000,NaN
3,Apcnt_1000076,3232.0,0.28505,2.8032,0.0000,0.0,0.0,473802.0,473802.0,1724437.0,...,NaN,0.916663,2.02,0.464224,NaN,NaN,charge,90.163742,0.788809,0.104029
4,Apcnt_1000080,3466.0,2.09545,0.8318,2.5182,0.0,19839.0,1150662.0,1150662.0,7860523.0,...,NaN,0.234047,23.23,0.726688,0.0,0.0,lending,1303.587148,1.637733,0.163124


### Encoding the categorical objects to be used

In [7]:
train["form_field47"] = LabelEncoder().fit_transform(train["form_field47"])
train["default_status"] = LabelEncoder().fit_transform(train["default_status"])
test["form_field47"] = LabelEncoder().fit_transform(test["form_field47"])

In [8]:
X= train.drop(["Applicant_ID", "default_status"], axis=1)
y= train["default_status"]
x_test= test.drop("Applicant_ID", axis=1) 

### Training and testing the models with hyperparameters

Running Hyperparameter testing with GridsearchCV was too computationally expensive and time consuming as I was using CPU, so I was limited in tuning my parameters

In [9]:
test_pred=[]
y_preds=[]
splits= None
fold= StratifiedKFold(n_splits=10, shuffle= True, random_state=2019)


for train_index, test_index in fold.split(X,y):
    xtrain, xval = X.iloc[train_index], X.iloc[test_index]
    ytrain, yval= y.iloc[train_index], y.iloc[test_index]
    
    model=CatBoostClassifier(silent=True, eval_metric="AUC", iterations= 2000, learning_rate=0.01, l2_leaf_reg=3.5,
                            depth=8, rsm=0.98, random_seed=2019)
    model.fit(xtrain, ytrain, eval_set=[(xtrain, ytrain),(xval, yval)], early_stopping_rounds=100,  verbose=True)
    predic= model.predict_proba(xval)[:,1]
    test_pred.append(predic)
    preds= model.predict_proba(x_test)[:,1]
    y_preds.append(preds)

0:	test: 0.8049679	test1: 0.7970518	best: 0.7970518 (0)	total: 195ms	remaining: 6m 29s
1:	test: 0.8140026	test1: 0.8076268	best: 0.8076268 (1)	total: 298ms	remaining: 4m 58s
2:	test: 0.8185479	test1: 0.8122007	best: 0.8122007 (2)	total: 406ms	remaining: 4m 29s
3:	test: 0.8202326	test1: 0.8135065	best: 0.8135065 (3)	total: 558ms	remaining: 4m 38s
4:	test: 0.8232300	test1: 0.8165064	best: 0.8165064 (4)	total: 682ms	remaining: 4m 32s
5:	test: 0.8242157	test1: 0.8174569	best: 0.8174569 (5)	total: 822ms	remaining: 4m 33s
6:	test: 0.8250711	test1: 0.8182682	best: 0.8182682 (6)	total: 959ms	remaining: 4m 33s
7:	test: 0.8257993	test1: 0.8185369	best: 0.8185369 (7)	total: 1.1s	remaining: 4m 33s
8:	test: 0.8260977	test1: 0.8188844	best: 0.8188844 (8)	total: 1.25s	remaining: 4m 35s
9:	test: 0.8265938	test1: 0.8190185	best: 0.8190185 (9)	total: 1.42s	remaining: 4m 42s
10:	test: 0.8265065	test1: 0.8188526	best: 0.8190185 (9)	total: 1.56s	remaining: 4m 43s
11:	test: 0.8262087	test1: 0.8185322	best: 

93:	test: 0.8359383	test1: 0.8268268	best: 0.8268268 (93)	total: 15.7s	remaining: 5m 18s
94:	test: 0.8359694	test1: 0.8268649	best: 0.8268649 (94)	total: 15.9s	remaining: 5m 19s
95:	test: 0.8360461	test1: 0.8268755	best: 0.8268755 (95)	total: 16.2s	remaining: 5m 21s
96:	test: 0.8361701	test1: 0.8270391	best: 0.8270391 (96)	total: 16.4s	remaining: 5m 22s
97:	test: 0.8361773	test1: 0.8270134	best: 0.8270391 (96)	total: 16.7s	remaining: 5m 24s
98:	test: 0.8362374	test1: 0.8270657	best: 0.8270657 (98)	total: 16.9s	remaining: 5m 23s
99:	test: 0.8363404	test1: 0.8271538	best: 0.8271538 (99)	total: 17s	remaining: 5m 23s
100:	test: 0.8364623	test1: 0.8272230	best: 0.8272230 (100)	total: 17.2s	remaining: 5m 23s
101:	test: 0.8365422	test1: 0.8273184	best: 0.8273184 (101)	total: 17.4s	remaining: 5m 23s
102:	test: 0.8365935	test1: 0.8273937	best: 0.8273937 (102)	total: 17.7s	remaining: 5m 26s
103:	test: 0.8366693	test1: 0.8274572	best: 0.8274572 (103)	total: 18s	remaining: 5m 27s
104:	test: 0.8367

184:	test: 0.8419497	test1: 0.8309878	best: 0.8309878 (184)	total: 36.3s	remaining: 5m 55s
185:	test: 0.8420209	test1: 0.8310478	best: 0.8310478 (185)	total: 36.5s	remaining: 5m 55s
186:	test: 0.8420746	test1: 0.8311035	best: 0.8311035 (186)	total: 36.6s	remaining: 5m 55s
187:	test: 0.8421170	test1: 0.8311475	best: 0.8311475 (187)	total: 36.8s	remaining: 5m 54s
188:	test: 0.8421774	test1: 0.8312272	best: 0.8312272 (188)	total: 37.1s	remaining: 5m 55s
189:	test: 0.8422593	test1: 0.8312591	best: 0.8312591 (189)	total: 37.3s	remaining: 5m 55s
190:	test: 0.8423079	test1: 0.8313150	best: 0.8313150 (190)	total: 37.5s	remaining: 5m 55s
191:	test: 0.8423684	test1: 0.8313327	best: 0.8313327 (191)	total: 37.7s	remaining: 5m 55s
192:	test: 0.8424034	test1: 0.8313355	best: 0.8313355 (192)	total: 38s	remaining: 5m 55s
193:	test: 0.8424585	test1: 0.8313665	best: 0.8313665 (193)	total: 38.2s	remaining: 5m 55s
194:	test: 0.8425154	test1: 0.8313838	best: 0.8313838 (194)	total: 38.5s	remaining: 5m 56s
1

275:	test: 0.8463071	test1: 0.8332221	best: 0.8332221 (275)	total: 59.2s	remaining: 6m 9s
276:	test: 0.8463453	test1: 0.8332457	best: 0.8332457 (276)	total: 59.5s	remaining: 6m 9s
277:	test: 0.8463806	test1: 0.8332652	best: 0.8332652 (277)	total: 59.7s	remaining: 6m 9s
278:	test: 0.8464388	test1: 0.8332799	best: 0.8332799 (278)	total: 1m	remaining: 6m 10s
279:	test: 0.8464659	test1: 0.8332743	best: 0.8332799 (278)	total: 1m	remaining: 6m 10s
280:	test: 0.8465096	test1: 0.8333339	best: 0.8333339 (280)	total: 1m	remaining: 6m 10s
281:	test: 0.8465694	test1: 0.8333577	best: 0.8333577 (281)	total: 1m	remaining: 6m 11s
282:	test: 0.8466396	test1: 0.8333640	best: 0.8333640 (282)	total: 1m 1s	remaining: 6m 11s
283:	test: 0.8466793	test1: 0.8333725	best: 0.8333725 (283)	total: 1m 1s	remaining: 6m 11s
284:	test: 0.8467356	test1: 0.8333668	best: 0.8333725 (283)	total: 1m 1s	remaining: 6m 12s
285:	test: 0.8467784	test1: 0.8333902	best: 0.8333902 (285)	total: 1m 2s	remaining: 6m 12s
286:	test: 0.8

365:	test: 0.8501624	test1: 0.8343651	best: 0.8343652 (364)	total: 1m 26s	remaining: 6m 24s
366:	test: 0.8502020	test1: 0.8343915	best: 0.8343915 (366)	total: 1m 26s	remaining: 6m 24s
367:	test: 0.8502381	test1: 0.8343977	best: 0.8343977 (367)	total: 1m 26s	remaining: 6m 24s
368:	test: 0.8502886	test1: 0.8344275	best: 0.8344275 (368)	total: 1m 26s	remaining: 6m 24s
369:	test: 0.8503435	test1: 0.8344418	best: 0.8344418 (369)	total: 1m 27s	remaining: 6m 24s
370:	test: 0.8503775	test1: 0.8344830	best: 0.8344830 (370)	total: 1m 27s	remaining: 6m 24s
371:	test: 0.8504084	test1: 0.8345051	best: 0.8345051 (371)	total: 1m 27s	remaining: 6m 24s
372:	test: 0.8504379	test1: 0.8345205	best: 0.8345205 (372)	total: 1m 28s	remaining: 6m 24s
373:	test: 0.8504726	test1: 0.8345131	best: 0.8345205 (372)	total: 1m 28s	remaining: 6m 24s
374:	test: 0.8505112	test1: 0.8345282	best: 0.8345282 (374)	total: 1m 28s	remaining: 6m 24s
375:	test: 0.8505598	test1: 0.8345224	best: 0.8345282 (374)	total: 1m 29s	remain

...

1638:	test: 0.8912156	test1: 0.8419959	best: 0.8420190 (1607)	total: 16m 16s	remaining: 3m 35s
1639:	test: 0.8912539	test1: 0.8419993	best: 0.8420190 (1607)	total: 16m 17s	remaining: 3m 34s
1640:	test: 0.8912739	test1: 0.8420007	best: 0.8420190 (1607)	total: 16m 17s	remaining: 3m 33s
1641:	test: 0.8913112	test1: 0.8419919	best: 0.8420190 (1607)	total: 16m 17s	remaining: 3m 33s
1642:	test: 0.8913426	test1: 0.8419934	best: 0.8420190 (1607)	total: 16m 18s	remaining: 3m 32s
1643:	test: 0.8913679	test1: 0.8419907	best: 0.8420190 (1607)	total: 16m 18s	remaining: 3m 31s
1644:	test: 0.8913897	test1: 0.8419964	best: 0.8420190 (1607)	total: 16m 19s	remaining: 3m 31s
1645:	test: 0.8914171	test1: 0.8420153	best: 0.8420190 (1607)	total: 16m 19s	remaining: 3m 30s
1646:	test: 0.8914446	test1: 0.8420081	best: 0.8420190 (1607)	total: 16m 20s	remaining: 3m 30s
1647:	test: 0.8914533	test1: 0.8420084	best: 0.8420190 (1607)	total: 16m 20s	remaining: 3m 29s
1648:	test: 0.8914904	test1: 0.8420090	best: 0.842

1725:	test: 0.8936641	test1: 0.8421446	best: 0.8421446 (1725)	total: 17m 2s	remaining: 2m 42s
1726:	test: 0.8936964	test1: 0.8421417	best: 0.8421446 (1725)	total: 17m 3s	remaining: 2m 41s
1727:	test: 0.8937161	test1: 0.8421536	best: 0.8421536 (1727)	total: 17m 3s	remaining: 2m 41s
1728:	test: 0.8937599	test1: 0.8421705	best: 0.8421705 (1728)	total: 17m 4s	remaining: 2m 40s
1729:	test: 0.8937791	test1: 0.8421743	best: 0.8421743 (1729)	total: 17m 5s	remaining: 2m 40s
1730:	test: 0.8938373	test1: 0.8421777	best: 0.8421777 (1730)	total: 17m 6s	remaining: 2m 39s
1731:	test: 0.8938949	test1: 0.8421674	best: 0.8421777 (1730)	total: 17m 6s	remaining: 2m 38s
1732:	test: 0.8939348	test1: 0.8421717	best: 0.8421777 (1730)	total: 17m 7s	remaining: 2m 38s
1733:	test: 0.8939739	test1: 0.8421696	best: 0.8421777 (1730)	total: 17m 8s	remaining: 2m 37s
1734:	test: 0.8940082	test1: 0.8421700	best: 0.8421777 (1730)	total: 17m 8s	remaining: 2m 37s
1735:	test: 0.8940247	test1: 0.8421725	best: 0.8421777 (1730

1812:	test: 0.8961989	test1: 0.8422551	best: 0.8422803 (1796)	total: 17m 59s	remaining: 1m 51s
1813:	test: 0.8962356	test1: 0.8422518	best: 0.8422803 (1796)	total: 18m	remaining: 1m 50s
1814:	test: 0.8962710	test1: 0.8422618	best: 0.8422803 (1796)	total: 18m	remaining: 1m 50s
1815:	test: 0.8963028	test1: 0.8422867	best: 0.8422867 (1815)	total: 18m 1s	remaining: 1m 49s
1816:	test: 0.8963308	test1: 0.8422941	best: 0.8422941 (1816)	total: 18m 1s	remaining: 1m 48s
1817:	test: 0.8963562	test1: 0.8422906	best: 0.8422941 (1816)	total: 18m 2s	remaining: 1m 48s
1818:	test: 0.8963935	test1: 0.8422975	best: 0.8422975 (1818)	total: 18m 3s	remaining: 1m 47s
1819:	test: 0.8964220	test1: 0.8422834	best: 0.8422975 (1818)	total: 18m 3s	remaining: 1m 47s
1820:	test: 0.8964430	test1: 0.8422868	best: 0.8422975 (1818)	total: 18m 4s	remaining: 1m 46s
1821:	test: 0.8964862	test1: 0.8422922	best: 0.8422975 (1818)	total: 18m 5s	remaining: 1m 46s
1822:	test: 0.8965146	test1: 0.8422984	best: 0.8422984 (1822)	tot

1899:	test: 0.8986100	test1: 0.8424317	best: 0.8424317 (1899)	total: 18m 56s	remaining: 59.8s
1900:	test: 0.8986438	test1: 0.8424372	best: 0.8424372 (1900)	total: 18m 57s	remaining: 59.2s
1901:	test: 0.8986657	test1: 0.8424373	best: 0.8424373 (1901)	total: 18m 57s	remaining: 58.6s
1902:	test: 0.8986888	test1: 0.8424422	best: 0.8424422 (1902)	total: 18m 58s	remaining: 58s
1903:	test: 0.8987062	test1: 0.8424442	best: 0.8424442 (1903)	total: 18m 59s	remaining: 57.4s
1904:	test: 0.8987340	test1: 0.8424575	best: 0.8424575 (1904)	total: 18m 59s	remaining: 56.8s
1905:	test: 0.8987612	test1: 0.8424608	best: 0.8424608 (1905)	total: 19m	remaining: 56.2s
1906:	test: 0.8987847	test1: 0.8424618	best: 0.8424618 (1906)	total: 19m 1s	remaining: 55.6s
1907:	test: 0.8988094	test1: 0.8424546	best: 0.8424618 (1906)	total: 19m 1s	remaining: 55.1s
1908:	test: 0.8988378	test1: 0.8424489	best: 0.8424618 (1906)	total: 19m 2s	remaining: 54.5s
1909:	test: 0.8988567	test1: 0.8424498	best: 0.8424618 (1906)	total: 

1987:	test: 0.9009806	test1: 0.8426875	best: 0.8426875 (1987)	total: 19m 54s	remaining: 7.21s
1988:	test: 0.9010118	test1: 0.8426920	best: 0.8426920 (1988)	total: 19m 54s	remaining: 6.61s
1989:	test: 0.9010394	test1: 0.8426987	best: 0.8426987 (1989)	total: 19m 55s	remaining: 6.01s
1990:	test: 0.9010715	test1: 0.8427051	best: 0.8427051 (1990)	total: 19m 56s	remaining: 5.41s
1991:	test: 0.9011115	test1: 0.8427192	best: 0.8427192 (1991)	total: 19m 57s	remaining: 4.81s
1992:	test: 0.9011411	test1: 0.8427106	best: 0.8427192 (1991)	total: 19m 57s	remaining: 4.21s
1993:	test: 0.9011799	test1: 0.8427085	best: 0.8427192 (1991)	total: 19m 58s	remaining: 3.6s
1994:	test: 0.9012231	test1: 0.8427042	best: 0.8427192 (1991)	total: 19m 58s	remaining: 3s
1995:	test: 0.9012388	test1: 0.8427147	best: 0.8427192 (1991)	total: 19m 59s	remaining: 2.4s
1996:	test: 0.9012534	test1: 0.8427225	best: 0.8427225 (1996)	total: 20m	remaining: 1.8s
1997:	test: 0.9012874	test1: 0.8427157	best: 0.8427225 (1996)	total: 2

In [10]:
y_preds = np.mean(y_preds, axis=0)
FinetunedModel= pd.DataFrame({"Applicant_ID":test["Applicant_ID"], "default_status": y_preds})
FinetunedModel.head()

,Applicant_ID,default_status
0,Apcnt_1000032,0.303131
1,Apcnt_1000048,0.413923
2,Apcnt_1000052,0.368069
3,Apcnt_1000076,0.751204
4,Apcnt_1000080,0.171852


In [11]:
FinetunedModel.to_csv("fine_tune.csv", index= False)